# 이시국 검사기

세상이 어쩌다 이지경이 됐을까?
2020년에는 마스크 끼면서 집에서 유튜브 보다가 다 지나게 생겼습니다.

화가 납니다.
다들 마스크 좀 잘 끼고 손 잘 씻고 다녔으면,
이 사단이 진작에 끝이 날 수 있었습니다.

그런데 뭐, 사람 사는 세상이 다 그렇지 않나요.
사람은 너무 약한 동물이고,
누구는 까먹어서 혹은 누구는 고의적으로 마스크를 끼지 않는 경우도 많습니다.

그런데 적어도, 제 주변에서는 이런 경우가 적었으면 좋겠습니다.
그래서 만들었습니다.
마스크를 착용하지 않은 사람들을 색출해내는,
이시국 검사기!

Copyright 2020. `Ho Kim`. All rights reversed, For education only.

## Setup

이시국 검사기는 딥러닝으로 만들 겁니다.
특히, 요즘에 성능이 잘나오는 [yolov5](https://github.com/ultralytics/yolov5)를 사용하겠습니다.

이 튜토리얼 또한, 이 사람들이 만든 아래의 튜토리얼을 따라 만들었습니다.
* https://github.com/ultralytics/yolov5/blob/master/tutorial.ipynb

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
!pip install -qr yolov5/requirements.txt  # install dependencies (ignore errors)
%cd yolov5

import torch
from IPython.display import Image, clear_output  # to display images
from utils.google_utils import gdrive_download  # to download models/datasets

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

## GPU 자원 검사

이시국 검사기는 성능 좋은 GPU를 사용해 학습하겠습니다.
아래의 연산을 수행한다면, Google Colab에서 사용 가능한 GPU의 목록이 표시됩니다.

만약, GPU가 제대로 표시되지 않다면,
메뉴의 Runtime-Change runtime type 에서 GPU를 선택해 다시 실행해주세요.

In [ ]:
!nvidia-smi

## 객체인식 테스트

일단, 딥러닝으로 객체인식을 하는 예시를 보여드리겠습니다.

In [ ]:
!python detect.py --weights yolov5s.pt --img 416 --conf 0.4 --source inference/images/
Image(filename='inference/output/zidane.jpg', width=600)

![성능 확실하구만](http://img.battlepage.com/upload/2020/0903/0320260191a07365fccc5aa6e25bfb0713171d98.png)

## 데이터셋 가져오기

마스크를 쓴 사람, 대충 쓴 사람, 쓰지 않은 사람의 사진 묶음이 담긴 데이터셋을 다운로드합니다.

아래의 kaggle 사이트에서 다운로드 후, 필요한 기능을 추가하였습니다.
* https://www.kaggle.com/andrewmvd/face-mask-detection

이 데이터셋은 해당 프로젝트 외의 사용을 금합니다.

In [ ]:
gdrive_download('1Ff6i4Ryz8xXYn5wKoSIP98-7QUZk79ZZ', 'masks.zip')

## 데이터셋 가공하기

`Yolov5` 가 사용 가능한 데이터 형식에 맞춰줍니다.

In [ ]:
!mkdir masks
!mv origin to_yolov5.py yolov5s_masks.yaml masks

!cd masks; python to_yolov5.py

yolov5/masks 디렉토리에 데이터셋이 구성되었습니다.

## 모델 학습하기

이제, `yolov5`에게 이시국 검사기를 학습시켜보겠습니다.
꽤 오래 걸릴 겁니다.

`Google Colab`은 12시간동안 무료 GPU를 제공하는 좋은 사이트입니다.
12시간이 지나더라도, 다시 접속해서 다시 실행하면 그만입니다.

다만, 90분동안 아무 행동이 없으면 세션이 종료됩니다.
즉, 이 딥러닝 학습이 90분 이상일 때, 나도 모르게 학습이 되다 마는 겁니다.

이 문제의 해결 방법이 아래의 사이트에 잘 나와있습니다.
* https://somjang.tistory.com/entry/Google-Colab-%EC%9E%90%EC%A3%BC%EB%81%8A%EA%B8%B0%EB%8A%94-%EB%9F%B0%ED%83%80%EC%9E%84-%EB%B0%A9%EC%A7%80%ED%95%98%EA%B8%B0

In [ ]:
!python train.py --img 640 --weights '' --epochs 300 --batch 64 --data masks/data.yaml --cfg masks/yolov5s_masks.yaml

## 성능 평가하기

이제, 학습한 이시국 검사기의 성능을 한번 보겠습니다~

In [ ]:
!cp runs/exp0/weights/best.pt yolov5s_masks.pt 
!python detect.py --source masks/images/val/maksssksksss104.png --weights yolov5s_masks.pt

![성능 확실하구만](http://img.battlepage.com/upload/2020/0903/0320260191a07365fccc5aa6e25bfb0713171d98.png)

## 모델 다운로드하기

마지막으로, 학습한 이시국 검사기 가중치 뭉치를 다운로드하겠습니다.

In [ ]:
from google.colab import files
files.download('yolov5s_masks.pt') 